In [1]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from enum import Enum
import pandas as pd


In [23]:
class Platform(Enum):
    REVERB = "reverb"
    VINTAGE = "vintage_and_rare"
    EBAY = "ebay"
    WEBSITE = "website"
    AMAZON = "amazon"
    WALMART = "walmart"


In [24]:
@dataclass
class ProductMapping:
    """
    Master product mapping class that maintains relationships between internal SKUs
    and platform-specific identifiers.
    """
    internal_sku: str
    description: str
    category: str
    platform_mappings: Dict[Platform, str] = field(default_factory=dict)
    attributes: Dict[str, str] = field(default_factory=dict)
    source_platform: Optional[Platform] = None
    source_data: Dict[str, any] = field(default_factory=dict)
    
    def add_platform_mapping(self, platform: Platform, platform_sku: str) -> None:
        """Add or update a platform-specific SKU mapping."""
        self.platform_mappings[platform] = platform_sku
    
    def get_platform_sku(self, platform: Platform) -> Optional[str]:
        """Get platform-specific SKU if it exists."""
        return self.platform_mappings.get(platform)
        

In [25]:
class ProductMappingSystem:
    """
    System to manage product mappings across different e-commerce platforms.
    """
    def __init__(self):
        self._mappings: Dict[str, ProductMapping] = {}
        self._platform_configs: Dict[Platform, Dict[str, str]] = {
            Platform.VINTAGE: {
                'sku_col': 'product id',
                'title_col': 'product model name',
                'brand_col': 'brand name',
                'category_col': 'category name',
                'description_col': 'product description',
                'price_col': 'product price'
            },
            Platform.AMAZON: {'sku_col': 'seller-sku', 'title_col': 'item-name'},
            Platform.EBAY: {'sku_col': 'Custom Label', 'title_col': 'Title'},
            Platform.WALMART: {'sku_col': 'sku', 'title_col': 'product_name'}
        }
    
    def add_product(self, mapping: ProductMapping) -> None:
        """Add a new product mapping."""
        self._mappings[mapping.internal_sku] = mapping
    
    def get_product(self, internal_sku: str) -> Optional[ProductMapping]:
        """Retrieve a product mapping by internal SKU."""
        return self._mappings.get(internal_sku)
    
    def import_platform_data(self, platform: Platform, data: pd.DataFrame, 
                           sku_column: str, mapping_column: str) -> None:
        """
        Import platform-specific SKU mappings from a DataFrame.
        
        Args:
            platform: The e-commerce platform
            data: DataFrame containing the mapping data
            sku_column: Column name containing internal SKUs
            mapping_column: Column name containing platform-specific SKUs
        """
        for _, row in data.iterrows():
            internal_sku = str(row[sku_column])
            platform_sku = str(row[mapping_column])
            
            if internal_sku not in self._mappings:
                # Create new mapping if it doesn't exist
                self._mappings[internal_sku] = ProductMapping(
                    internal_sku=internal_sku,
                    description="",  # Can be updated later
                    category=""      # Can be updated later
                )
            
            self._mappings[internal_sku].add_platform_mapping(platform, platform_sku)
    
    def export_platform_mappings(self, platform: Platform) -> pd.DataFrame:
        """Export all mappings for a specific platform to a DataFrame."""
        data = []
        for mapping in self._mappings.values():
            platform_sku = mapping.get_platform_sku(platform)
            if platform_sku:
                data.append({
                    'internal_sku': mapping.internal_sku,
                    'platform_sku': platform_sku,
                    'description': mapping.description,
                    'category': mapping.category
                })
        return pd.DataFrame(data)

In [29]:
def generate_internal_sku(row: pd.Series, platform: Platform, sku_column: str) -> str:
    """Generate an internal SKU from platform data using a hash of identifying fields."""
    identifying_fields = []
    
    if platform == Platform.VINTAGE:
        identifying_fields = [
            str(row[sku_column]),
            str(row['product model name']),
            str(row['brand name'])
        ]
    # Keep AMAZON in enum but don't need config yet
    
    return f"INT-{hash(''.join(identifying_fields)) & 0xFFFFFF:06X}"

def process_platform_files(file_paths: Dict[Platform, str]) -> ProductMappingSystem:
    mapping_system = ProductMappingSystem()
    
    platform_configs = {
        Platform.VINTAGE: {
            'sku_col': 'product id',
            'title_col': 'product model name',
            'brand_col': 'brand name',
            'category_col': 'category name'
        }
    }
    
    for platform, file_path in file_paths.items():
        config = platform_configs[platform]
        df = pd.read_csv(file_path)
        
        for _, row in df.iterrows():
            internal_sku = generate_internal_sku(row, platform, config['sku_col'])
            
            if internal_sku not in mapping_system._mappings:
                mapping_system.add_product(ProductMapping(
                    internal_sku=internal_sku,
                    description=row[config['title_col']],
                    category=row[config['category_col']]
                ))
            
            mapping_system._mappings[internal_sku].add_platform_mapping(
                platform, 
                str(row[config['sku_col']])
            )
    
    return mapping_system


def example_usage():
    # Initialize the mapping system
    mapping_system = ProductMappingSystem()

    # Create a sample VINTAGE product mapping
    vintage_data = pd.DataFrame({
        'product id': [1001, 1002],
        'product model name': ['Fender Stratocaster', 'Gibson Les Paul'],
        'brand name': ['Fender', 'Gibson'],
        'category name': ['Electric Guitars', 'Electric Guitars'],
        'product description': ['1965 Strat', '1959 Burst'],
        'product price': [15000, 250000]
    })

    mapping_system.import_platform_data(
        Platform.VINTAGE,
        vintage_data,
        'product id',
        'product id'
    )

    # Export mappings
    vintage_mappings = mapping_system.export_platform_mappings(Platform.VINTAGE)
    return vintage_mappings


In [ ]:
# from product_mapping import ProductMappingSystem, Platform

# Initialize system
mapping_system = ProductMappingSystem()

# Define your CSV files
files = {
    # Platform.REVERB: 'Reverb_Mass_Upload.csv',
    Platform.VINTAGE: 'vintageandrare_inventory_2025-01-21_09-28.csv',
    # Platform.EBAY: 'walmart_download.csv'
}

# Process all files
mapping_system = process_platform_files(files)

# Review the mappings
for internal_sku, mapping in mapping_system._mappings.items():
    print(f"\nInternal SKU: {internal_sku}")
    print(f"Description: {mapping.description}")
    print("Platform SKUs:", mapping.platform_mappings)

# Export specific platform mappings
vintage_df = mapping_system.export_platform_mappings(Platform.VINTAGE)
vintage_df.to_csv('vintage_mappings.csv', index=False)

In [28]:
import pandas as pd
vintage_df = pd.read_csv('vintageandrare_inventory_2025-01-21_09-28.csv')
vintage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   brand name              972 non-null    object 
 1   category name           972 non-null    object 
 2   external link           28 non-null     object 
 3   product id              972 non-null    int64  
 4   external id             0 non-null      float64
 5   product model name      972 non-null    object 
 6   product price           972 non-null    int64  
 7   product price notax     0 non-null      float64
 8   product description     971 non-null    object 
 9   product year            952 non-null    float64
 10  product sold            972 non-null    object 
 11  product finish          951 non-null    object 
 12  product in collective   972 non-null    object 
 13  product in inventory    972 non-null    object 
 14  product in reseller     972 non-null    ob